In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

from sklearn.decomposition import IncrementalPCA, MiniBatchNMF
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score, make_scorer

import random
import pickle
import pydicom
import matplotlib.pyplot as plt


import data_splitter as ds
import data_loader as dl
import feature_extraction as fex
import run_model as rm

In [2]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV, HalvingRandomSearchCV

## Image Resolution 256 x 256

In [4]:
X = np.load('sample_flatten_X.npy')
y = np.load('sample_flatten_y.npy')

In [5]:
from collections import Counter
Counter(y)

Counter({0.0: 8570, 1.0: 181})

In [7]:
nmf = MiniBatchNMF(
    max_iter = 100000,
    tol = 1e-4, #control early stopping based on changes in H
    max_no_improvement = 3, #control early stopping based on smoothed cost function
    forget_factor = 1, #rescaling factor of past
    fresh_restarts = False, #completely solve for W at each step 
    random_state = 42
    )

logit = LogisticRegression(
    warm_start = True,
    solver = 'saga',
    max_iter = 20,
    l1_ratio = 0.5,
    random_state = 42
    )

pipe = Pipeline([
    ('feature_extraction', nmf),
    ('classify', logit)
])

param_grid = {
    'feature_extraction__n_components': np.arange(2, 51, step=1),
    'feature_extraction__init': ['random', 'nndsvd', 'nndsvda'],
    'feature_extraction__beta_loss': ['frobenius', 'kullback-leibler'],
    'feature_extraction__alpha_W': np.arange(0, 1, step=0.1), #np.logspace(0, -4, num=5, base=10, endpoint=True),
    'feature_extraction__l1_ratio': np.arange(0, 1, step=0.1)
}

In [8]:
RANDOM_STATE = 256

grid = HalvingRandomSearchCV(
    pipe,
    cv = StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True),
    param_distributions = param_grid,
    
    factor = 5, # 1/5 candidates selected in each iteration
    aggressive_elimination = True,
    n_candidates = 'exhaust',
    min_resources = 400,
    scoring = make_scorer(average_precision_score, needs_proba=True),
    refit = False,
    
    n_jobs = -1,
    random_state = 42,
    verbose = 2
)
    
grid.fit(X, y)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

...

[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=19; total time=  23.6s


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=19; total time= 4.6min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=19; total time= 4.9min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=19; total time= 5.1min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=16; total time= 3.1min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extr

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

HalvingRandomSearchCV(aggressive_elimination=True,
                      cv=StratifiedKFold(n_splits=5, random_state=256, shuffle=True),
                      estimator=Pipeline(steps=[('feature_extraction',
                                                 MiniBatchNMF(forget_factor=1,
                                                              max_iter=100000,
                                                              max_no_improvement=3,
                                                              random_state=42)),
                                                ('classify',
                                                 LogisticRegression(l1_ratio=0.5,
                                                                    max_iter=20,
                                                                    random_state=42,
                                                                    solver='saga',
                                                                    warm_start=True))]),
                      f...
                                           'feature_extraction__l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                           'feature_extraction__n_components': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])},
                      random_state=42, refit=False,
                      scoring=make_scorer(average_precision_score, needs_proba=True),
                      verbose=2)

In [9]:
print(grid.best_params_)

{'feature_extraction__n_components': 36, 'feature_extraction__l1_ratio': 0.5, 'feature_extraction__init': 'nndsvd', 'feature_extraction__beta_loss': 'frobenius', 'feature_extraction__alpha_W': 0.0}


In [10]:
print(grid.best_score_)

0.03512903804190755


In [11]:
import joblib

#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_256_minre400__round8_randomstate256.pkl')

['models/nmf_hp_search_256_minre400__round8_randomstate256.pkl']

[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=16; total time=15.8min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=43; total time=54.3min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=18; total time=42.9min
[CV] END feature_extraction__alpha_W=0.7000000000000001, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=17; total time= 4.5min
[CV] END feature_extraction__alpha_W=0.2, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, featur

## Image Resolution 512 x 512

In [6]:
X = np.load('/scratch/siads699w23_class_root/siads699w23_class/irinb/sample_flatten_X_512.npy')
y = np.load('/scratch/siads699w23_class_root/siads699w23_class/irinb/sample_flatten_y_512.npy')

In [7]:
from collections import Counter
Counter(y)

Counter({0.0: 8570, 1.0: 181})

In [8]:
nmf = MiniBatchNMF(
    max_iter = 100000,
    tol = 1e-4, #control early stopping based on changes in H
    max_no_improvement = 3, #control early stopping based on smoothed cost function
    forget_factor = 1, #rescaling factor of past
    fresh_restarts = False, #completely solve for W at each step 
    random_state = 42
    )

logit = LogisticRegression(
    warm_start = True,
    solver = 'saga',
    max_iter = 20,
    l1_ratio = 0.5,
    random_state = 42
    )

pipe = Pipeline([
    ('feature_extraction', nmf),
    ('classify', logit)
])

param_grid = {
    'feature_extraction__n_components': np.arange(2, 31, step=1),
    'feature_extraction__init': ['random', 'nndsvd', 'nndsvda'],
    'feature_extraction__beta_loss': ['frobenius', 'kullback-leibler'],
    'feature_extraction__alpha_W': np.arange(0, 1, step=0.1), #np.logspace(0, -4, num=5, base=10, endpoint=True),
    'feature_extraction__l1_ratio': np.arange(0, 1, step=0.1)
}

In [ ]:
RANDOM_STATE = 42

grid = HalvingRandomSearchCV(
    pipe,
    cv = StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True),
    param_distributions = param_grid,
    
    factor = 5, # 1/5 candidates selected in each iteration
    aggressive_elimination = True,
    n_candidates = 'exhaust',
    min_resources = 400,
    scoring = make_scorer(average_precision_score, needs_proba=True),
    refit = False,
    
    n_jobs = -1,
    random_state = 42,
    verbose = 2
)
    
grid.fit(X, y)

n_iterations: 2
n_required_iterations: 2
n_possible_iterations: 2
min_resources_: 400
max_resources_: 8751
aggressive_elimination: True
factor: 5
----------
iter: 0
n_candidates: 21
n_resources: 400
Fitting 5 folds for each of 21 candidates, totalling 105 fits


/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/s

...

[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=21; total time=23.2min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=17; total time= 1.6min
[CV] END feature_extraction__alpha_W=0.9, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.2, feature_extraction__n_components=17; total time=15.2min
[CV] END feature_extraction__alpha_W=0.0, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.6000000000000001, feature_extraction__n_components=17; total time=24.2min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.30000000000

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=frobenius, feature_extraction__init=nndsvda, feature_extraction__l1_ratio=0.9, feature_extraction__n_components=5; total time=22.8min
[CV] END feature_extraction__alpha_W=0.6000000000000001, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=random, feature_extraction__l1_ratio=0.1, feature_extraction__n_components=26; total time=46.9min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=frobenius, feature_extraction__init=random, feature_extraction__l1_ratio=0.30000000000000004, feature_extraction__n_components=6; total time=29.7min
[CV] END feature_extraction__alpha_W=0.30000000000000004, feature_extraction__beta_loss=kullback-leibler, feature_extraction__init=nndsvd, feature_extraction__l1_ratio=0.0, feature_extraction__n_components=18; total time= 9.7min
[CV] END feature_extraction__alpha_W=0.1, feature_extraction__beta_loss=kullback-leib

/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/home/irinb/.local/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


----------
iter: 1
n_candidates: 5
n_resources: 2000
Fitting 5 folds for each of 5 candidates, totalling 25 fits


In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
import joblib

#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_512_round1.pkl')

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
import joblib

#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_512_round2.pkl')

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_512_round3.pkl')

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_512_round4.pkl')

## Image Resolution 1024 x 1024

In [ ]:
BASEPATH = '/scratch/siads699w23_class_root/siads699w23_class/shared_data/team_mammogram/train_images'

# scitkit-learn parameter search does not support batch learning
# so we'll have to combine data in 1 batch
BATCH_SIZE = 8751 

# from PCA, image resolution does not seem to have any effect on no. of component
# so we'll use the lowest resolution to minimize training cost
IMG_SIZE = (1024, 1024, 1)

RANDOM_STATE = 42

# define DataGenerators for training split
train_gen = dl.DataGenerator(
    list_IDs = sample_img_ids,
    labels = splitter.labels,
    patient_img_dict = splitter.trainset,
    basepath = BASEPATH,
    batch_size = BATCH_SIZE,
    img_size = IMG_SIZE,
    n_classes = 2,
    shuffle = True,
    normalize = (0, 1),
    feature_extractor = None,
    verbose = True
    )

# check no. of batch
print(len(train_gen))

In [ ]:
# get X and y
for X, y in train_gen:
  X = X.reshape(BATCH_SIZE, -1)
  y = y[:, 1]
  print(X.shape, y.shape)

In [ ]:
with open('/scratch/siads699w23_class_root/siads699w23_class/irinb/sample_flatten_X_1024.npy','wb') as f:
    np.save(f, X)
    
with open('/scratch/siads699w23_class_root/siads699w23_class/irinb/sample_flatten_y_1024.npy','wb') as f:
    np.save(f, y)

In [ ]:
Counter(y)

In [ ]:
nmf = MiniBatchNMF(
    max_iter = 100000,
    tol = 1e-4, #control early stopping based on changes in H
    max_no_improvement = 3, #control early stopping based on smoothed cost function
    forget_factor = 1, #rescaling factor of past
    fresh_restarts = False, #completely solve for W at each step 
    random_state = 42
    )

logit = LogisticRegression(
    warm_start = True,
    solver = 'saga',
    max_iter = 20,
    l1_ratio = 0.5,
    random_state = 42
    )

pipe = Pipeline([
    ('feature_extraction', nmf),
    ('classify', logit)
])

param_grid = {
    'feature_extraction__n_components': np.arange(2, 31, step=1),
    'feature_extraction__init': ['random', 'nndsvd', 'nndsvda'],
    'feature_extraction__beta_loss': ['frobenius', 'kullback-leibler'],
    'feature_extraction__alpha_W': np.arange(0, 1, step=0.1), #np.logspace(0, -4, num=5, base=10, endpoint=True),
    'feature_extraction__l1_ratio': np.arange(0, 1, step=0.1)
}

In [ ]:
RANDOM_STATE = 42

grid = HalvingRandomSearchCV(
    pipe,
    cv = StratifiedKFold(n_splits=5, random_state=RANDOM_STATE, shuffle=True),
    param_distributions = param_grid,
    
    factor = 5, # 1/5 candidates selected in each iteration
    aggressive_elimination = True,
    n_candidates = 'exhaust',
    scoring = make_scorer(average_precision_score, needs_proba=True),
    refit = False,
    
    n_jobs = -1,
    random_state = 42,
    verbose = 2
)
    
grid.fit(X, y)

In [ ]:
print(grid.best_params_)

In [ ]:
print(grid.best_score_)

In [ ]:
#save your model or results
joblib.dump(grid, 'models/nmf_hp_search_1024.pkl')